# Uncertainty Engine SDK example workflows - train and predict a 1D model

This notebook goes through how you would set up a workflow to train and save a machine-learning model using the `TrainModel` node and then make predictions on new data using the `PredictModel` node.

Start by importing and initializing the `Client`. 

In [1]:
import os
os.environ["UE_ACCOUNT_ID"]
from uncertainty_engine import Client, Environment
from amentum_librti.config.settings import ACCOUNT_ID

client = Client()
client.authenticate(ACCOUNT_ID)

KeyError: 'UE_ACCOUNT_ID'

With useful imports:

In [ ]:
from pprint import pprint
import pandas as pd
import os

from amentum_librti.plotting.plots import plot_1d
from amentum_librti.utils.helper import get_presigned_url

cwd = os.getcwd()
repo_root = os.path.dirname(cwd)
data_directory = repo_root+"/src/amentum_librti/data/"  # Change this to your data directory

## Part 1: Building and running a **train** workflow

### Viewing the nodes

Once you have initialised the Uncertainty Engine client you can use the `list_nodes` method to find your node information. Fields such as the `description`, `inputs` and `outputs` are particularly useful for building your workflow.

In [ ]:
# Create a dictionary with node IDs as keys
nodes = client.list_nodes()
nodes_by_id = {node["id"]: node for node in nodes}

# Print the details of the `TrainModel` node
pprint(nodes_by_id["TrainModel"])

Now we know that the `TrainModel` node needs the following inputs:
- **Input Dataset:** Train input data
- **Output Dataset:** Target output data
- **Model Config:** A model config node specifying the parameters we wish to use for training

### Creating our train and test data

In this example we will use some synthetic data to demonstrate how it can be used to train a model. 
We need two datasets, one with fewer data points (to train) and one with more data points (to test).

The data can be generated via https://librti-toy.streamlit.app/

Produce the datasets, and save your data to a known directory.

In [ ]:
train_filename = 'train_corrosion_dataset.csv'
test_filename = 'test_corrosion_dataset.csv'

Now we will load these as a dataframe.

In [ ]:
train_df = pd.read_csv(data_directory+train_filename)
test_df = pd.read_csv(data_directory+test_filename)

Let's check which parameters we have.

In [ ]:
print(train_df.columns.tolist())

### Configuring our model

As we can see by the `TrainModel` node info, it needs a `ModelConfig`. To define this ModelConfig we need a ModelConfig node. Like for the TrainModel node we can examine the inputs and outputs of the ModelConfig node.

In [ ]:
# Print `ModelConfig` node details
pprint(nodes_by_id["ModelConfig"])

The `ModelConfig` node accepts several optional parameters that control model training. However, as none of the inputs are required we can just use the default input parameters for now. This will configure our model as a `SingleTaskGP` which is ideal for our regression task. A `SingleTaskGP` (Single Task Gaussian Process) models the relationship between a single input and output variable while providing uncertainty estimates alongside predictions.

### Constructing a workflow

First, import and initialise the `Graph` class.


In [ ]:
from uncertainty_engine.graph import Graph

# Create a new graph
train_graph = Graph()

In [ ]:
from uncertainty_engine.nodes.base import Node

# Define the model config node
model_config = Node(
    node_name="ModelConfig",
    label="Model Config",
)

# Add node to the graph and connect it to the train node
train_graph.add_node(model_config)

# Add a handle to the the config output
output_config = model_config.make_handle("config")

Then we can start adding nodes to the graph. The first node we will add is the `TrainModel` node, with our train `x` and `y` CSVs as inputs.

In [ ]:
# Convert X values to CSV string
input_csv = train_df[['temperature_c']].to_csv(index=False)

# Convert Y values to CSV string
output_csv = train_df[['comp_Fe']].to_csv(index=False)

# Create the train node with the data as input
train_model = Node(
    node_name="TrainModel",
    label="Train Model",
    config=output_config,
    inputs={"csv": input_csv},
    outputs={"csv": output_csv}
)

# Add the node to the graph
train_graph.add_node(train_model)

### Assigning an output node

Now our graph has been built we can use an output node to decide how we wish to collect the output. The output nodes we could use here are:
- **Download:** To download the model JSON using an AWS pre-signed url.
- **Save:** To save the model JSON to the Uncertainty Engine cloud.

First lets make a handle for the output `model` of the `Train` node.

In [ ]:
output_model = train_model.make_handle("model")

In this case we will use the `Download` node to get a pre-signed URL for our trained model so we can use it for the `Predict` part of this notebook.

In [ ]:
# Define the Download node
download = Node(
  node_name="Download",
  label="Download",
  file=output_model
)

# Add the download node to the graph and connect it to the train node
train_graph.add_node(download)

# Print graph to check the structure
pprint(train_graph.nodes)

### Executing a workflow

Create the executable workflow by wrapping our graph in the `Workflow` node and defining the `requested_output` as the output handle of the `Download` node.

In [ ]:
from uncertainty_engine.nodes.workflow import Workflow

# Define our output handles
output_download_model = download.make_handle("file")

# Wrap the graph in a workflow node
train_workflow = Workflow(
    graph=train_graph.nodes,
    input=train_graph.external_input,
    external_input_id=train_graph.external_input_id,
    requested_output={
        "Trained Model": output_download_model.model_dump(),
        }
    )

Now, we can execute the workflow by running `client.run_node(workflow)` and passing the workflow object. This may take a minute for training.

In [ ]:
train_response = client.run_node(train_workflow)

Following the presigned URL will download the trained model. In the following cell we use this link to define our `model_json` so that it can be used for our `predict` workflow.

In [ ]:
# Get the presigned URL from the response
trained_model_url = train_response.outputs["outputs"]["Trained Model"]



# Get the model from the presigned URL
model_response = get_presigned_url(trained_model_url)
model_json = model_response.json()

## Part 2: Building and running a **predict** workflow

Next, we will use the saved model json to make predictions on a sample dataset. If we take a look at the `PredictModel` node information we can see that the following inputs are needed to run the node:
- **Dataset:** The input data on which to make the output predictions (this will be our `test_df` dataset)
- **Model:** The pre-trained model to be used to make the predictions
- **Seed (optional):** A random seed for reproducibility

In [ ]:
# Print `PredictModel` node details
pprint(nodes_by_id["PredictModel"])

Now we know what input parameters the `PredictModel` node needs we can start constructing our workflow.

### Constructing the workflow

Let's create a new instance of the `Graph` class (as we don't wish to build on top of our train workflow) and add our predict node using our local `trained_model.json` as the `model` and the `x` column of our `test_df` as the `dataset` we wish to make predictions on.

In [ ]:
# Initialise the predict graph
predict_graph = Graph()

# Convert test values to CSV string
test_input_csv = test_df[['temperature_c']].to_csv(index=False)

# Define predict node with test data as input dataset and model json as model
predict = Node(
    node_name="PredictModel",
    label="Predict",
    dataset={"csv": test_input_csv},
    model=model_json,
    seed=42,
)

# Add the predict node to the graph
predict_graph.add_node(predict)

We can use the `Download` node to get our resulting datasets. We will need one `Download` node for both the predictions and uncertainty output.

In [ ]:
# Add handles to the prediction and uncertainty outputs
output_predictions = predict.make_handle("prediction")
output_uncertainty = predict.make_handle("uncertainty")

# Define download nodes for predictions and uncertainty
download_predictions = Node(
    node_name="Download",
    label="Download Predictions",
    file=output_predictions
)
download_uncertainty = Node(
    node_name="Download",
    label="Download Uncertainty",
    file=output_uncertainty
)

# Add download nodes to the graph and connect them to the predict node
predict_graph.add_node(download_predictions)
predict_graph.add_node(download_uncertainty)

# Print the predict graph
pprint(predict_graph.nodes)

### Executing the workflow

Now we can run our workflow using the `Workflow` node.

In [ ]:
# Define the output handles for the download nodes
output_download_predictions = download_predictions.make_handle("file")
output_download_uncertainty = download_uncertainty.make_handle("file")

# Wrap the predict graph in a workflow node
predict_workflow = Workflow(
    graph=predict_graph.nodes,
    input=predict_graph.external_input,
    external_input_id=predict_graph.external_input_id,
    requested_output={
        "Predictions": output_download_predictions.model_dump(),
        "Uncertainty": output_download_uncertainty.model_dump(),
        }
    )

# Run the predict workflow and get the response
predict_response = client.run_node(predict_workflow)

We can download the results and save the `predictions` and `uncertainty` as pandas dataframes so that we can plot them.

In [ ]:
from io import StringIO

# Download the predictions and save as a DataFrame
predictions_response = get_presigned_url(predict_response.outputs["outputs"]["Predictions"])
predictions_df = pd.read_csv(StringIO(predictions_response.text))  # Save the predictions to a DataFrame

# Download the uncertainty and save as a DataFrame
uncertainty_response = get_presigned_url(predict_response.outputs["outputs"]["Uncertainty"])
uncertainty_df = pd.read_csv(StringIO(uncertainty_response.text))  # Save the uncertainty to a DataFrame

### Plot the results

Finally, we can plot our predictions vs our truth function. 

In [ ]:
fig, ax = plot_1d(
    train_df=train_df,
    test_df=test_df,
    predictions_df=predictions_df, 
    uncertainty_df=uncertainty_df, 
    x_col="temperature_c",
    y_col="comp_Fe",
    credible_level=0.99,
    title="",
    x_label="Temperature (°C)",
    y_label="Fe composition (mass fraction)",
    save_path=None,  # or "figs/fe_gp.png"
)

### Conclusion

Qualitatively we can see that the model predictions closely follow the ground truth. The plot also shows the uncertainty on the model predictions. As is typical for a Gaussian Process model, the uncertainty is generally larger in areas with sparse training samples.

In this notebook we have demonstrated how to use the SDK to train a model and then use it to make predictions. 

If you wanted to lead on from this and get a more accurate reading of how well the model performed you could try using the `ScoreModel` node to evaluate the model. Otherwise, checkout [`demo_resource.ipynb`](./demo_resource.ipynb) to learn about how you can use resources in the Uncertainty Engine SDK.